In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [11]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")
r.clean()

2023/04/11 11:41:20 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'mlflow-recipes-titanic' with profile: 'local'


In [12]:
# Display the DAGs (Directed Acyclic Graph)
# r.inspect()

In [13]:
r.run("ingest")

2023/04/11 11:41:24 INFO mlflow.recipes.step: Running step ingest...
/tmp/tmp2p_l3kqx/train.csv
custom


name,type
Survived,integer
Pclass,integer
Name,string
Sex,string
Age,number
SibSp,integer
Parch,integer
Ticket,string
Fare,number
Cabin,string


In [ ]:
r.run("split")

In [ ]:
training_data = r.get_artifact("training_data")
training_data.describe()

In [ ]:
r.run("transform")

In [ ]:
r.run("train")

In [ ]:
trained_model = r.get_artifact("model")
print(trained_model)

In [ ]:
r.run("evaluate")

In [ ]:
r.run("register")